In [22]:
import os
from data import read_examples
from utils import print_example

data_path = './RACE/'
train_high, train_middle = read_examples(os.path.join(data_path, 'train'))

print_example(train_high, 0)

id: 
 high1.txt
passage: 
 My husband is a born shopper. He loves to look at things and to touch them. He likes to compare prices between the same items in different shops. He would never think of buying anything without looking around in several different shops. On the other hand, I'm not a shopper. I think shopping is boring and unpleasant. If I like something and I have enough money to take it, I buy it at once. I never look around for a good price or a better deal. Of course my husband and I never go shopping together. Doing shopping together would be too painful for both of us. When it comes to shopping, we go our different ways.
Sometimes I ask my son Jimmy to buy some food in the shop not far from our home. But he is always absent-minded. This was his story.
One day I said to him, " I hope you won't forget what I have told you to buy." " No," said Jimmy. "I won't forget. You want three oranges , six eggs and a pound of meat."
He went running down the street to the shop. As he ra

In [ ]:
from preprocessing import dcmn_preprocess

sample_size = 10000
sample_data = train_high[:1000]
validation_data = train_high[sample_size:sample_size + 100]

sample_inputs, sample_answers = dcmn_preprocess(sample_data)
valid_inputs, valid_answers = dcmn_preprocess(validation_data)
answer_dict = {'A': tf.constant([1, 0, 0, 0]),
               'B': tf.constant([0, 1, 0, 0]),
               'C': tf.constant([0, 0, 1, 0]),
               'D': tf.constant([0, 0, 0, 1])}
sample_answers_encoded = tf.convert_to_tensor([answer_dict[a] for a in sample_answers])
valid_answers_encoded = tf.convert_to_tensor([answer_dict[a] for a in valid_answers])